# 支持向量机

## 一、 什么是SVM?

支持向量机(Support Vector Machine,SVM)是用于分类的一种算法，也属于有监督学习的范畴.                
<font color='red'>概述一下:</font>              
当一个分类问题，数据是线性可分(linearly separable)的，也就是用一根棍就可以将两种小球分开的时候，我们 只要将棍的位置放在让小球距离棍的距离最大化的位置即可，寻找这个最大间隔的过程，就叫做<font color='red'>最优化</font>。但是，现实往往是很残酷的，一般的数据是线性不可分的，也就是找不到一个棍将两种小球很好的分类。这个时候，我们就 需要像大侠一样，将小球拍起，用一张纸代替小棍将小球进行分类。想要让数据飞起，我们需要的东西就是<font color='red'>核函数(kernel)</font>，用于切分小球的纸，就是<font color='red'>超平面(hyperplane)</font>。<font color='green'>如果数据集是N维的，那么超平面就是N-1维的。</font>

<font color='red'>具有“最大间隔”的超平面就是SVM要寻找的最优解</font>。而这个真正的最优解对应的两侧虚线所穿过的样本点，就是SVM中的<font color='red'>支持样本点</font>，称为“支持向量(support vector)”。支持向量到超平面的距离被称为<font color='red'>间隔(margin)</font>。

## 二、线性SVM

一个最优化问题通常有两个最基本的因素: 
- 1) 目标函数，也就是你希望什么东西的什么指标达到最好; 
- 2) 优化对象，你期望通过改变哪些因素来使你的目标函数达到最优。           

在线性SVM算法中，目标函数显然就是那个“间隔”，而优化对象则是超平面。我们以线性可分的二分类问题为例。

### 1.超平面方程

在线性可分的二分类问题中，超平面其实就是一条直线。相信直线方程大家都不陌生:
$y=ax+b$(公式1)            
现在我们做个小小的改变，让原来的 轴变成 轴， 变成 轴，于是公式(1)中的直线方程会变成下面的样子:             
$x_2=ax_1+b$(公式2)        
$ax_1+(-1)x_2+b=0$(公式3)               
向量形式写成：

\begin{align}
(a+b)^{\prime}&=(BA)^{\prime}=B((AB)^{\prime})^2A\nonumber\\
&=\left(
\begin{array}{cc}
b & 1\\
\end{array}
\right)
&\left(
\begin{array}{cc}
b^{\prime} & z\\
0 & a^{\prime}\\
\end{array}
\right)^2
&
\left(
\begin{array}{c}
1 \\
a \\
\end{array}
\right)\nonumber\\
&=b^{\prime}+bb^{\prime}za+bza^{\prime}a+a^{\prime}\nonumber\\
&=(1-bb^{\prime})[\sum\limits_{i=0}^sb^i(a^{\prime})^{i+1}]+[\sum\limits_{i=0}^r(b^{\prime})^{i+1}a^i](1-aa^{\prime})\nonumber\\
&=(1-bb^{\prime})[\sum\limits_{i=0}^{s-1}b^i(a^{\prime})^{i+1}]+[\sum\limits_{i=0}^{r-1}(b^{\prime})^{i+1}a^i](1-aa^{\prime}) \nonumber \qquard \#
\end{align}

## 三、SMO算法

### 1. 什么是SMO算法

SMO算法就是序列最小优化(Sequential Minimal Optimization)，它是由 John Platt 于1996年发布的专门用于 训练SVM的一个强大算法。SMO算法的目的是将大优化问题分解为多个小优化问题来求解。这些小优化问题往往 很容易求解，并且对它们进行顺序求解的结果与将它们作为整体来求解的结果完全一致的。在结果完全相同的同 时，SMO算法的求解时间短很多。

SMO算法的目标是求出一系列$\alpha$和b，一旦求出了这些，就很容易计算出权重向量$w$并得到分隔超平面。              
SMO算法的工作原理是:每次循环中选择两个 进行优化处理。一旦找到了一对合适的 ，那么就增大其中一个同 时减小另一个。这里所谓的"合适"就是指两个$\alpha$必须符合以下两个条件:
 * 两个$\alpha$必须要在间隔边界之外
 * 这两个$\alpha$还没有进行过区间化处理或者不在边界上。

### 2. SMO算法流程

### 3. 简化版SMO算法

#### 3.1 SMO算法的伪代码

- 创建一个α向量并初始化为0向量 
- 当迭代次数 < 最大迭代次数时<font color='red'>(外循环)</font>:
    - 对数据集中每个数据向量<font color='red'>(内循环)</font>: 
        - 如果该数据向量可以被优化:
             * 随机选择另外一个数据向量
             * 同时优化这两个向量
             * 如果两个向量都不能被优化，则退出内循环
    - 如果所有向量都没被优化，迭代次数+1，继续下一次循环

#### 3.2 构建辅助函数         
生成特征向量和标签向量

In [19]:
import numpy as np
import pandas as pd
""" 
函数功能:
    创建特征向量和标签向量 
参数说明:
    file:原始文件路径 
返回:
    xMat:特征向量
    yMat:标签向量 
"""
def loadDataSet(file):
   dataSet= pd.read_table(file,header = None)
   xMat=np.mat(dataSet.iloc[:,:-1].values)
   yMat=np.mat(dataSet.iloc[:,-1].values).T
   return xMat,yMat

数据可视化

In [20]:
import matplotlib.pyplot as plt
%matplotlib inline

def showDataSet(xMat, yMat):
    data_p = []  #正样本
    data_n = []   #负样本
    m = xMat.shape[0] #样本总数
    for i in range(m):
        if yMat[i] > 0:
            data_p.append(xMat[i])
        else:
            data_n.append(xMat[i])
    data_p_ = np.array(data_p)   #转换为numpy矩阵 
    data_n_ = np.array(data_n)    #转换为numpy矩阵 
    plt.scatter(data_p_.T[0], data_p_.T[1])   #正样本散点图
    plt.scatter(data_n_.T[0], data_n_.T[1])   #负样本散点图
    plt.show()

随机选择$\alpha$对：

In [21]:
import random
""" 
函数功能:
    随机选择一个索引 
参数说明:
    i:第一个alpha索引
    m:数据集总行数 
返回:
    j:随机选择的不与i相等的值 
"""
def selectJrand(i,m):
    j=i
    while (j==i):
        j=int(random.uniform(0,m))
    return j

$\alpha_j$的修剪函数

In [22]:
""" 
函数功能:
    修剪alpha_j 
"""
def clipAlpha(aj,H,L):
    if aj>H:
        aj=H
    if L>aj:
        aj=L
    return aj

#### 3.3 简化版SMO算法

In [29]:
"""
函数功能: 
参数说明:
    xMat:特征向量 
    yMat:标签向量
    C:常数
    toler:容错率 
    maxIter:最大迭代次数
返回: 
    b、alpha
"""
def smoSimple(xMat,yMat,C,toler,maxIter):
    b=0  #初始化b参数
    m,n=xMat.shape    #m为数据集的总行数，n为特征的数量
    alpha = np.mat(np.zeros((m,1)))   #初始化alpha参数，设为0 
    iters =0   #初始化迭代次数
    while (iters<maxIter):
        alpha_ = 0   #初始化alpha优化次数
        for i in range(m):
            #步骤1:计算误差Ei 
            fXi=np.multiply(alpha,yMat).T*(xMat*xMat[i,:].T)+b 
            Ei=fXi-yMat[i]
            #优化alpha，设定容错率
            if ((yMat[i]*Ei<-toler)and(alpha[i]<C)) or((yMat[i]*Ei>toler)and(alpha[i]>0)): 
                #随机选择一个与alpha_i成对优化的alpha_j
                j=selectJrand(i,m)
                #步骤1:计算误差Ej 
                fXj=np.multiply(alpha,yMat).T*(xMat*xMat[j,:].T)+b 
                Ej=fXj-yMat[j]
                #保存更新前的alpha_i和alpha_j 
                alphaIold=alpha[i].copy() 
                alphaJold=alpha[j].copy()
                #步骤2:计算上下界H和L
                if (yMat[i]!=yMat[j]):
                    L=max(0,alpha[j]-alpha[i])
                    H=min(C,C+alpha[j]-alpha[i])
                else:
                    L=max(0,alpha[j]+alpha[i]-C)
                    H=min(C,C+alpha[j]+alpha[i])
                if L==H:
                    #print('L==H')
                    continue
                    #步骤3:计算学习率eta(eta是alpha_j的最优修改量) 
                    eta=2*xMat[i,:]*xMat[j,:].T-xMat[i,:]*xMat[i,:].T-xMat[j,:]*xMat[j,:].T 
                    if eta>=0:
                        #print('eta>=0')
                        continue
                    #步骤4:更新alpha_j
                    alpha[j]-= yMat[j]*(Ei-Ej)/eta #步骤5:修剪alpha_j
                    alpha[j]=clipAlpha(alpha[j],H,L)
                    if abs(alpha[j]-alphaJold)<0.00001:
                        #print('alpha_j 变化太小')
                        continue
                    #步骤6:更新alpha_i
                    alpha[i]+=yMat[j]*yMat[i]*(alphaJold-alpha[j])
                    #步骤7:更新b_1和b_2 
                    b1=b-Ei-yMat[i]*(alpha[i]-alphaIold)*xMat[i,:]*xMat[i,:].T-yMat[j]*(alpha[j]-alphaJold)*xMat[i,:]*xMat[j,:].T
                    b2=b-Ej-yMat[i]*(alpha[i]-alphaIold)*xMat[i,:]*xMat[j,:].T-yMat[j]*(alpha[j]-alphaJold)*xMat[j,:]*xMat[j,:].T 
                    #步骤8:根据b_1和b_2更新b
                    if (0<alpha[i])and(C>alpha[i]): 
                        b=b1 
                    elif (0<alpha[j])and(C>alpha[j]): 
                        b=b2 
                    else: 
                        b=(b1+b2)/2
                    #统计优化次数
                    alpha_+=1
                    #print(f'第{iters}次迭代 样本{i},alpha优化次数:{alpha_}') 
            #更新迭代次数
            if alpha_==0: 
                iters+=1 
            else: 
                iters=0
                #print(f'迭代次数为:{iters}')
        return b,alpha

In [30]:
%time b,alpha=smoSimple(xMat,yMat,0.6,0.001,5)

NameError: name 'xMat' is not defined